<a href="https://colab.research.google.com/github/YasJanam/NLP_MODELS_2/blob/main/DataAugmentation_4/DataAugmentation_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**DataAugmentation**

**Dataset : manu/wnut_17**

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from datasets import load_dataset
from transformers import AutoTokenizer
import pandas as pd

In [10]:
dataset = load_dataset("manu/wnut_17")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/926 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/309k [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/67.1k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/116k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3394 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1009 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1287 [00:00<?, ? examples/s]

In [11]:
dataset = dataset.remove_columns(['id'])

In [12]:
# Dataset classes
id2label ={'0': "O",
            '1': "B-corporation",
            '2': "I-corporation",
            '3': "B-creative-work",
            '4': "I-creative-work",
            '5': "B-group",
            '6': "I-group",
            '7': "B-location",
            '8': "I-location",
            '9': "B-person",
            '10': "I-person",
            '11': "B-product",
            '12': "I-product"}

label2id = {k:v for v,k in id2label.items()}

In [17]:
trn_data = dataset['train']

##### **📌 توضیح کلاس‌ها:**
###### 🟥**person** → اسم آدم‌ها (مثل: Elon Musk, Taylor Swift)
###### 🟪**location** → مکان‌ها (شهر، کشور، منطقه، آدرس؛ مثل: New York, Mount Everest)
###### 🟩**corporation** → شرکت‌ها و سازمان‌ها (مثل: Google, United Nations)
###### 🟨**product** → محصولات (مثل: iPhone 15, Coca-Cola)
###### 🟦**group** → گروه‌ها و تیم‌ها (مثل: The Beatles, Manchester United)
###### 🟧**creative-work** → آثار هنری یا رسانه‌ای (مثل: Harry Potter, Game of Thrones, Star Wars)


#### **DataAugmentation**

###### **entity_replacement** :

این کلاس از جایگزینی نام ها در جملات دیتاست برای افزایش داده استفاده میکند

یعنی در هر جمله، نام ها را تغییر میدهد و جمله جدید را به دیتاست می افزاید

مثلا اگر در یک جمله  نام ها را 5 بار عوض کند، 5 جمله جدید به دیتاست اضافه میشود

###### **patterns_based_generation** :    

انجام میدهدentity_replacement ،این کلاس روی یک لیست قالب های جملات

جملات این لیست، جملات دیتاست اصلی نیستند!! این جملات را در لیست های پایین میبینید

هر جمله یک قالب است که با تغییر نام های آن جمله میتوان جملات جدید تولید کرد و انها را به لیست اضافه کرد

دو لیست قالب جملات داریم. یکی از انها جملات ساده تر است و دیگری جملات مرکب و طولانی تر

###### **لیست اسامی:**
corporation

creative_work

groups

locations

persons

products

چند تا لیست هم داریم که از اون لیست ها اسم انتخاب میکنیم

لیستها پایین هستند

شیوه انتخاب اسم رندوم است


In [24]:
import random
from datasets import Dataset

class entity_replacement(object):
  def __init__(self,dataset,num=6):
    self.num_samples = num  # از هر جمله موجود چند جمله جدید بسازیم
    self.dataset = dataset

  def get_sample(self,ner_tag):
    if ner_tag == 1 : # corporation
      len_list = len(corporation)
      index =  random.randint(0, len_list - 1)
      samp = corporation[index]
      return samp

    if ner_tag == 3 : # creative_work
      len_list = len(creative_work)
      index = random.randint(0, len_list - 1)
      samp = creative_work[index]
      return samp

    if ner_tag == 5 : # groups
      len_list = len(groups)
      index = random.randint(0, len_list - 1)
      samp = groups[index]
      return samp

    if ner_tag == 7 : # locations
      len_list = len(locations)
      index = random.randint(0, len_list - 1)
      samp = locations[index]
      return samp

    if ner_tag == 9 : # persons
      len_list = len(persons)
      index = random.randint(0, len_list - 1)
      samp = persons[index]
      return samp

    if ner_tag == 11 : # products
      len_list = len(products)
      index = random.randint(0, len_list - 1)
      samp = products[index]
      return samp

    return None # Return None for other tags

  def new_item(self,item):
    count = 0
    while(count < len(item['tokens'])):
      tag = item['ner_tags'][count]
      next_tag = item['ner_tags'][count + 1] if count + 1 < len(item['ner_tags']) else None
      if tag != 0 :
        if next_tag == 0 :
          item['tokens'][count] = self.get_sample(tag) # Corrected: Use self.get_sample
      count += 1
    return item

  def create_new_items(self,item):
    new_items = []
    for i in range(self.num_samples):
      processed_item = self.new_item(item) # Corrected: Use self.new_item
      new_items.append(processed_item)
    return new_items

  def new_dataset(self):
    new_dataset = []
    for item in self.dataset:
      new_items = self.create_new_items(item) # Corrected: Pass item
      new_dataset.extend(new_items)
    dataset = Dataset.from_list(new_dataset)
    return dataset

In [14]:
class pattern_based_generation(object):
  def __init__(self, templates_list,num):
    self.templates_list = templates_list
    #self.entity_replacement = entity_replacement(dataset,num)
    self.num_samples = num

  def convert_text_to_item(self,text):
    tokens = text.split()
    ner_tags = [0] * len(tokens)
    count = 0
    while(count < len(tokens)):
      if tokens[count] == '{person}':
        ner_tags[count] = 9
      elif tokens[count] == '{corporation}':
        ner_tags[count] = 1
      elif tokens[count] == '{group}':
        ner_tags[count] = 5
      elif tokens[count] == '{location}':
        ner_tags[count] = 7
      elif tokens[count] == '{product}':
        ner_tags[count] = 11
      elif tokens[count] == '{creative-work}':
        ner_tags[count] = 3
      count += 1
    item = {}
    item['tokens'] = tokens
    item['ner_tags'] = ner_tags
    return item

  def conver_list_items_to_dataset(self):
    data = {'tokens': [], 'ner_tags': []}
    for item in self.templates_list:
      processed_item = self.convert_text_to_item(item)
      data['tokens'].append(processed_item['tokens'])
      data['ner_tags'].append(processed_item['ner_tags'])
    dataset = Dataset.from_dict(data)
    return dataset

  def create_dataset(self):
      dataset_templates = self.conver_list_items_to_dataset()
      ent_rep = entity_replacement(dataset_templates,self.num_samples)
      new_dataset_template = ent_rep.new_dataset()
      return new_dataset_template

In [15]:
def cocat_datasets(dataset_list):
  final_dataset = {'tokens':[],'ner_tag':[]}
  for dataset in dataset_list:
    for item in dataset:
      final_dataset['tokens'].append(item['tokens'])
      final_dataset['ner_tag'].append(item['ner_tags'])
  final_dataset = Dataset.from_dict(final_dataset)
  return final_dataset

In [18]:
ent_rep = entity_replacement(trn_data,6)
train_data = ent_rep.new_dataset()
train_data

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 20364
})

In [19]:
pat_gen1 = pattern_based_generation(all_templates,8)
temp_dataset1 = pat_gen1.create_dataset()
temp_dataset1

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 1040
})

In [22]:
pat_gen2 = pattern_based_generation(more_complex_templates,13)
temp_dataset2 = pat_gen2.create_dataset()
temp_dataset2

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 1131
})

In [23]:
dataset_lists = [train_data,temp_dataset1,temp_dataset2]
final_dataset = cocat_datasets(dataset_lists)
final_dataset

Dataset({
    features: ['tokens', 'ner_tag'],
    num_rows: 22535
})

#### **Names & Templates**

In [8]:
products = [
    # محصولات الکترونیکی و تکنولوژی
    "iPhone",
    "iPad",
    "MacBook",
    "Surface",
    "Galaxy",
    "PlayStation",
    "Xbox",
    "Kindle",
    "Pixel",
    "AirPods",

    # نوشیدنی‌ها
    "CocaCola",
    "Pepsi",
    "Fanta",
    "Sprite",
    "RedBull",
    "Monster",
    "Nescafe",
    "Lipton",
    "Evian",
    "Heineken",

    # خوراکی‌ها و تنقلات
    "Oreo",
    "KitKat",
    "Snickers",
    "M&M",
    "Pringles",
    "Nutella",
    "PeanutButter",
    "Lay's",
    "Doritos",
    "Haribo",

    # محصولات مد و پوشاک
    "NikeAir",
    "AdidasUltra",
    "PumaShoes",
    "LevisJeans",
    "RayBan",
    "GucciBag",
    "LouisVuitton",
    "ChanelPerfume",
    "HermesScarf",
    "RolexWatch"
]


In [7]:
persons = [
    # دانشمندان و مخترعان
    "Einstein",
    "Newton",
    "Tesla",
    "Curie",
    "Galileo",
    "DaVinci",
    "Hawking",
    "Turing",
    "Feynman",
    "Edison",

    # هنرمندان و نویسندگان
    "Shakespeare",
    "Hemingway",
    "Poe",
    "Orwell",
    "Picasso",
    "VanGogh",
    "Klimt",
    "Dali",
    "Michelangelo",
    "Raphael",

    # شخصیت‌های تاریخی و سیاسی
    "Gandhi",
    "Lincoln",
    "Churchill",
    "Mandela",
    "Napoleon",
    "Cleopatra",
    "Alexander",
    "Caesar",
    "Washington",
    "KingJr",

    # ورزشکاران و افراد مشهور مدرن
    "Messi",
    "Ronaldo",
    "SerenaWilliams",
    "MichaelJordan",
    "LeBronJames",
    "TigerWoods",
    "RogerFederer",
    "UsainBolt",
    "TomBrady",
    "SimoneBiles",

    # بازیگران و شخصیت‌های فرهنگی
    "Beyonce",
    "Adele",
    "BradPitt",
    "AngelinaJolie",
    "LeonardoDiCaprio",
    "TomHanks",
    "EmmaWatson",
    "ScarlettJohansson",
    "DwayneJohnson",
    "WillSmith"
]


In [6]:
locations = [
    # کشورهای معروف
    "UnitedStates",
    "Canada",
    "Brazil",
    "Germany",
    "France",
    "Italy",
    "Spain",
    "UnitedKingdom",
    "Japan",
    "China",
    "India",
    "Australia",
    "Mexico",
    "Russia",
    "Egypt",
    "SouthAfrica",
    "Argentina",
    "Thailand",
    "Turkey",
    "Netherlands",

    # شهرهای معروف
    "NewYork",
    "LosAngeles",
    "Chicago",
    "London",
    "Paris",
    "Berlin",
    "Rome",
    "Madrid",
    "Tokyo",
    "Beijing",
    "Moscow",
    "Sydney",
    "Toronto",
    "Dubai",
    "Istanbul",

    # مکان‌های دیدنی و معروف
    "EiffelTower",
    "StatueOfLiberty",
    "GreatWall",
    "TajMahal",
    "Colosseum",
    "BigBen",
    "Pyramids",
    "MachuPicchu",
    "ChristTheRedeemer",
    "GoldenGateBridge",
    "Santorini",
    "NiagaraFalls",
    "MountEverest",
    "GrandCanyon",
    "SagradaFamilia"
]


In [5]:
groups = [
    # تیم‌ها و گروه‌های ورزشی
    "Barcelona",
    "RealMadrid",
    "ManchesterUnited",
    "Liverpool",
    "Juventus",
    "BayernMunich",
    "ChicagoBulls",
    "LA Lakers",
    "GoldenStateWarriors",
    "NewYorkYankees",
    "DallasCowboys",
    "AllBlacks",
    "PittsburghSteelers",
    "TorontoMapleLeafs",

    # گروه‌ها و بندهای موسیقی
    "Beatles",
    "Queen",
    "Coldplay",
    "LinkinPark",
    "ImagineDragons",
    "Metallica",
    "RollingStones",
    "Maroon5",
    "OneDirection",
    "ABBA",

    # گروه‌ها و استودیوهای فیلم و تلویزیون
    "Pixar",
    "WarnerBros",
    "MarvelStudios",
    "DCComics",
    "Paramount",
    "Universal",
    "DreamWorks",
    "StudioGhibli",
    "BBCStudios",

    # گروه‌های هنری و فرهنگی
    "Bauhaus",
    "Fluxus",
    "Dada",
    "Impressionists",
    "Surrealists",
    "Renaissance",
    "Postmodernists",
    "AbstractExpressionists",
    "Cubists",
    "PopArtists"
]


In [4]:
corporation = [
    # شرکت‌های فناوری و نرم‌افزاری
    "Google",
    "Microsoft",
    "Apple",
    "Amazon",
    "Facebook",
    "IBM",
    "Oracle",
    "Intel",
    "Adobe",
    "Salesforce",
    "Spotify",
    "Snapchat",
    "Twitter",
    "Tesla",
    "Nvidia",
    "Uber",
    "Airbnb",
    "Dropbox",
    "Slack",
    "Zoom",
    "Atlassian",
    "Shopify",
    "GitHub",
    "RedHat",
    "LinkedIn",
    "PayPal",

    # شرکت‌های تجاری و برندهای شناخته‌شده
    "Nike",
    "Adidas",
    "CocaCola",
    "Pepsi",
    "Starbucks",
    "Disney",
    "Netflix",
    "Sony",
    "Samsung",
    "Toyota",
    "BMW",
    "Intel",
    "Huawei",
    "LG",
    "McDonalds",
    "KFC",
    "Unilever",
    "ProcterGamble",
    "Nestle",
    "Volkswagen",
    "Heineken",
    "Ikea",
    "H&M",
    "Zara",
    "Lego",
    "Patagonia"
]


In [3]:
creative_work = [
    # نقاشی‌ها و مجسمه‌ها
    "StarryNight",
    "MonaLisa",
    "Persistence",
    "Scream","Guernica",
    "PearlEarring","KissKlimt","NightWatch","Venus","Creation","GreatWave","Demoiselles","Athens","LasMeninas","Gothic","EarthlyDelights","Arnolfini","HayWain",
    "Temeraire","Shalott","SonOfMan","KissBrancusi","Thinker","Burghers","KissRodin",

    # فیلم‌ها
    "Inception",
    "Titanic",
    "Avatar",
    "Gladiator",
    "Casablanca",
    "Joker",
    "Amelie",
    "Interstellar",
    "Parasite",
    "Frozen",

    # سریال‌ها
    "Friends",
    "BreakingBad",
    "GameOfThrones",
    "Sherlock",
    "StrangerThings",
    "Westworld",
    "Dexter",
    "Lost",
    "Vikings",

    # کتاب‌ها و داستان‌ها
    "HarryPotter",
    "LordOfTheRings",
    "PrideAndPrejudice",
    "Hobbit",
    "1984",
    "Dune",
    "MobyDick",
    "Gatsby",
    "Dracula",
    "Frankenstein"
]


In [2]:
all_templates = [
    # ===== PERSON =====
    "{person} works at {corporation}.",
    "{person} traveled to {location}.",
    "{person} lives in {location}.",
    "{person} is a member of {group}.",
    "{person} won an award for {creative-work}.",
    "{person} invented {product}.",
    "{person} wrote {creative-work}.",
    "{person} designed {product}.",
    "{person} joined {corporation} as CEO.",
    "{person} is the leader of {group}.",
    "{person} founded {corporation}.",
    "{person} recommended {product}.",
    "{person} directed {creative-work}.",
    "{person} created {product} with {corporation}.",
    "{person} gave a speech in {location}.",
    "{person} performed with {group}.",
    "{person} collaborated with {corporation} on {creative-work}.",
    "{person} was born in {location}.",
    "{person} studied at {location}.",
    "{person} supports {group}.",
    "{person} criticized {corporation}.",
    "{person} published {creative-work} through {corporation}.",
    "{person} announced {product} in {location}.",
    "{person} organized an event with {group}.",
    "{person} appeared in {creative-work}.",
    "{person} worked for {corporation} in {location}.",
    "{person} launched {product} at {corporation}.",
    "{person} reviewed {creative-work}.",
    "{person} collaborated with {person} on {creative-work}.",
    "{person} visited {corporation} in {location}.",

    # ===== CORPORATION =====
    "{corporation} announced a new project.",
    "{corporation} opened a branch in {location}.",
    "{corporation} produces {product}.",
    "{corporation} collaborated with {group} on a campaign.",
    "{corporation} is competing with {corporation}.",
    "{corporation} published {creative-work}.",
    "{corporation} hired {person}.",
    "{corporation} released {creative-work} in {location}.",
    "{corporation} is based in {location}.",
    "{corporation} sponsors {group}.",
    "{corporation} launched {product} last year.",
    "{corporation} invested in {group}.",
    "{corporation} signed a contract with {person}.",
    "{corporation} funded {creative-work}.",
    "{corporation} acquired {corporation}.",
    "{corporation} built an office in {location}.",
    "{corporation} tested {product} with {group}.",
    "{corporation} developed {creative-work} with {person}.",
    "{corporation} presented {product} in {location}.",
    "{corporation} partnered with {corporation} to release {product}.",

    # ===== GROUP =====
    "{group} organized a concert in {location}.",
    "The leader of {group} is {person}.",
    "{group} was founded in 1990.",
    "{group} competed with {group}.",
    "{group} released {creative-work}.",
    "{group} uses {product}.",
    "{group} performed in {location}.",
    "{group} collaborated with {corporation}.",
    "{group} recruited {person}.",
    "{group} protested against {corporation}.",
    "{group} supported {person}.",
    "{group} traveled to {location}.",
    "{group} created {creative-work}.",
    "{group} bought {product}.",
    "{group} presented {creative-work} in {location}.",
    "{group} featured {person} in {creative-work}.",
    "{group} launched {product} with {corporation}.",
    "{group} participated in {creative-work}.",
    "{group} campaigned in {location}.",
    "{group} honored {person} for {creative-work}.",

    # ===== LOCATION =====
    "{location} is a popular tourist spot.",
    "{corporation} opened a new office in {location}.",
    "{group} held a meeting in {location}.",
    "{creative-work} is set in {location}.",
    "{product} is sold in {location}.",
    "{person} gave a talk in {location}.",
    "{corporation} is headquartered in {location}.",
    "{person} traveled to {location} with {group}.",
    "{corporation} launched {product} in {location}.",
    "{group} performed {creative-work} in {location}.",
    "{person} studied at a university in {location}.",
    "{corporation} invested in {location}.",
    "{product} was tested in {location}.",
    "{creative-work} premiered in {location}.",
    "{group} organized a march in {location}.",
    "{person} visited {location} during holidays.",
    "{corporation} built a factory in {location}.",
    "{group} opened an office in {location}.",
    "{creative-work} was filmed in {location}.",
    "{person} was honored in {location}.",

    # ===== PRODUCT =====
    "{product} is manufactured by {corporation}.",
    "Many customers bought {product} in {location}.",
    "{person} recommended {product}.",
    "{product} is more expensive than {product}.",
    "The latest version of {product} was released by {corporation}.",
    "{creative-work} features {product}.",
    "{person} designed {product}.",
    "{group} uses {product} in their work.",
    "{product} is used in {creative-work}.",
    "{corporation} sells {product} in {location}.",
    "{person} bought {product} from {corporation}.",
    "{group} tested {product}.",
    "{product} was released in {location}.",
    "{product} supports {creative-work}.",
    "{person} promoted {product}.",
    "{corporation} donated {product} to {group}.",
    "{product} was reviewed in {creative-work}.",
    "{group} adopted {product}.",
    "{corporation} showcased {product} at {location}.",
    "{person} developed {product} at {corporation}.",

    # ===== CREATIVE-WORK =====
    "{creative-work} was released last year.",
    "{creative-work} was created by {group}.",
    "The director of {creative-work} is {person}.",
    "{creative-work} won an award.",
    "{corporation} published {creative-work}.",
    "{person} reviewed {creative-work}.",
    "{group} performed {creative-work}.",
    "{creative-work} was filmed in {location}.",
    "{creative-work} was inspired by {creative-work}.",
    "{creative-work} stars {person}.",
    "{creative-work} is about {group}.",
    "{creative-work} is sold in {location}.",
    "{creative-work} was banned by {group}.",
    "{creative-work} features {product}.",
    "{corporation} financed {creative-work}.",
    "{person} directed {creative-work}.",
    "{group} produced {creative-work}.",
    "{creative-work} was presented in {location}.",
    "{creative-work} competed with {creative-work}.",
    "{person} won a prize for {creative-work}.",
]

print("Number of templates:", len(all_templates))


Number of templates: 130


In [1]:
more_complex_templates = [
    # ==== Compound with conjunctions ====
    "{person} wrote {creative-work}, and {corporation} decided to publish it in {location}.",
    "{group} performed {creative-work}, but {corporation} refused to finance it.",
    "{person} invented {product}, so {corporation} promoted it worldwide.",
    "{corporation} supported {group}, yet {person} opposed their decision.",
    "{creative-work} won an award, and {person} thanked {group} during the ceremony.",

    # ==== Passive voice ====
    "{product} was designed by {person} and later improved by {corporation}.",
    "{creative-work} was reviewed by {group} before being released in {location}.",
    "{corporation} was founded by {person} in {location}.",
    "{group} was sponsored by {corporation} for their {creative-work}.",
    "{person} was awarded for {creative-work} in {location}.",

    # ==== Relative clauses ====
    "{person}, who previously worked at {corporation}, now leads {group}.",
    "{product}, which was launched by {corporation}, became popular in {location}.",
    "{creative-work}, which was inspired by {product}, was directed by {person}.",
    "{corporation}, whose headquarters is in {location}, released {creative-work}.",
    "{group}, which includes {person}, collaborated with {corporation}.",

    # ==== Time expressions ====
    "Last year, {person} created {product} in partnership with {corporation}.",
    "In 2020, {corporation} launched {creative-work} in {location}.",
    "For many years, {group} has been performing in {location}.",
    "In the future, {corporation} will expand to {location}.",
    "Recently, {person} reviewed {creative-work} for {corporation}.",

    # ==== Conditional variants ====
    "If {corporation} releases {product}, {group} will adopt it in {location}.",
    "As long as {person} supports {group}, {creative-work} will continue.",
    "Even if {product} fails, {corporation} will try again in {location}.",
    "If {creative-work} is successful, {person} will win an award.",
    "Provided that {corporation} invests in {group}, {product} will improve.",

    # ==== Comparisons ====
    "{person} is more influential in {group} than {person} in {corporation}.",
    "{creative-work} gained more attention in {location} than {creative-work}.",
    "{product} is cheaper in {location} than in {location}.",
    "{corporation} hired more employees than {corporation} last year.",
    "{group} produced more {creative-work} than {group}.",

    # ==== Questions ====
    "Who is responsible for {product} at {corporation}?",
    "What inspired {person} to write {creative-work}?",
    "When was {corporation} founded in {location}?",
    "Which {group} organized the event in {location}?",
    "Why did {corporation} acquire {product} from {group}?",
    "How did {person} collaborate with {corporation} on {creative-work}?",
    "Where did {group} perform {creative-work} last year?",
    "Whose {product} was presented by {corporation}?",
    "Which {creative-work} did {person} direct in {location}?",
    "What role did {corporation} play in {creative-work}?",

    # ==== Miscellaneous complex ====
    "After {person} presented {product}, {corporation} decided to invest.",
    "Before {group} released {creative-work}, {person} edited it carefully.",
    "Since {corporation} built its office in {location}, {product} sales increased.",
    "Because {person} joined {group}, {creative-work} was completed faster.",
    "Whenever {corporation} launches {product}, {group} provides support.",
    "While {group} was performing {creative-work}, {corporation} announced {product}.",
    "During the conference in {location}, {person} criticized {corporation}.",
    "At the same time that {person} directed {creative-work}, {corporation} developed {product}.",
    "By the time {creative-work} was published, {group} had already performed it in {location}.",
    "As soon as {corporation} released {product}, {person} recommended it.",

    # ==== Compound / Complex Sentences ====
    "Although {person} worked at {corporation}, {person} later joined {group} in {location}.",
    "{creative-work}, which was directed by {person}, was released by {corporation} in {location}.",
    "During the festival in {location}, {person} presented {creative-work} produced by {group}.",
    "{product}, originally designed by {person}, is now manufactured by {corporation} in {location}.",
    "{person}, a well-known member of {group}, introduced {product} during the event in {location}.",
    "{creative-work}, praised by critics, was financed by {corporation}.",
    "{corporation}, which competes with {corporation}, acquired {product} from {group}.",
    "After {person} left {corporation}, {group} invited them to {location}.",
    "{group} released {creative-work}, while {corporation} prepared {product} for launch.",
    "Even though {corporation} supported {group}, {person} opposed the partnership.",

    # ==== Conditional Sentences ====
    "If {person} accepts the offer from {corporation}, {product} will be developed in {location}.",
    "Unless {group} supports {creative-work}, {corporation} will cancel the project.",
    "If {corporation} expands to {location}, {group} will gain access to {product}.",
    "Should {person} publish {creative-work}, {corporation} will distribute it worldwide.",
    "If {product} fails in {location}, {corporation} might replace it with {product}.",

    # ==== Questions ====
    "Who founded {corporation} in {location}?",
    "When did {group} release {creative-work}?",
    "Which {product} was introduced by {corporation}?",
    "Why did {person} leave {corporation} for {group}?",
    "How many times has {group} performed {creative-work} in {location}?",
    "Where was {creative-work} filmed by {corporation}?",
    "What role did {person} play in {creative-work} produced by {group}?",

    # ==== Comparative Sentences ====
    "{product} is more popular in {location} than in {location}.",
    "{creative-work} directed by {person} received better reviews than {creative-work}.",
    "{corporation} has more employees in {location} than {corporation}.",
    "{group} is older than {group}.",
    "{person} sold more {product} than {person} in {location}.",

    # ==== Different Tenses ====
    "{person} had been working at {corporation} before joining {group}.",
    "{corporation} will launch {product} in {location} next year.",
    "{group} has been performing {creative-work} in {location} for years.",
    "{person} will introduce {product} after completing {creative-work}.",
    "{creative-work} had already been reviewed by {person} before {corporation} released it.",

    # ==== Descriptive / Appositive ====
    "{person}, the founder of {corporation}, attended the ceremony in {location}.",
    "{corporation}, a global leader, announced {product} in {location}.",
    "{creative-work}, a bestseller, was adapted into a film by {group}.",
    "{product}, known for its design, was improved by {person}.",
    "{group}, famous for {creative-work}, collaborated with {corporation}.",
]
